In [1]:
video_path = "input3.mp4"

In [ ]:
import av
import json
from av.sidedata.motionvectors import MotionVectors

# 1) Apri il container e prendi lo stream video
container = av.open(video_path)
stream = container.streams.video[0]

# 2) Imposta l'opzione per esportare i MV
stream.codec_context.options = {"flags2": "+export_mvs"}

all_flows = []

# 3) Demux e decodi
for packet in container.demux(stream):
    for frame in packet.decode():
        # 4) Itera su tutti i side-data e cattura i MotionVectors
        for sd in frame.side_data:
            if isinstance(sd, MotionVectors):
                ts = float(frame.pts * frame.time_base)
                vectors = [{
                    "source": mv.source,
                    "w": mv.w,
                    "h": mv.h,
                    "src_x": mv.src_x,
                    "src_y": mv.src_y,
                    "dst_x": mv.dst_x,
                    "dst_y": mv.dst_y,
                    "motion_x": mv.motion_x,
                    "motion_y": mv.motion_y,
                    "motion_scale": mv.motion_scale
                } for mv in sd]
                all_flows.append({"time": ts, "motion_vectors": vectors})

# 5) Salva in JSON
with open("mvs.json", "w") as f:
    json.dump(all_flows, f, indent=2)

print(f"Estratti MV da {len(all_flows)} frame, salvati in mvs.json")


Estratti MV da 122 frame, salvati in mvs.json


In [ ]:
import av
import json
from av.sidedata.sidedata import Type

# 1) Apri il container e prendi lo stream video
container = av.open(video_path)
stream = container.streams.video[0]

# 2) Imposta l'opzione per esportare i MV
stream.codec_context.options = {"flags2": "+export_mvs"}

all_flows = []

# 3) Demux e decodi
for packet in container.demux(stream):
    for frame in packet.decode():

        vectors = []
        ts = float(frame.pts * frame.time_base)

        mv_data = frame.side_data.get(Type.MOTION_VECTORS)
        if mv_data:
            for mv in mv_data:
                vectors.append({
                    "source": mv.source,
                    "w": mv.w,
                    "h": mv.h,
                    "src_x": mv.src_x,
                    "src_y": mv.src_y,
                    "dst_x": mv.dst_x,
                    "dst_y": mv.dst_y,
                    "motion_x": mv.motion_x,
                    "motion_y": mv.motion_y,
                    "motion_scale": mv.motion_scale
                })
        
        all_flows.append({"time": ts, "motion_vectors": vectors})

# 5) Salva in JSON
with open("mvs.json", "w") as f:
    json.dump(all_flows, f, indent=2)

print(f"Estratti MV da {len(all_flows)} frame, salvati in mvs.json")


Estratti MV da 123 frame, salvati in mvs.json
